# This notebook is demonstrating (and testing) scenarios for generating requests to the AICI server (evolving API)

In [1]:
import sys
#sys.path.insert(0, 'c:\\users\\emrek\\source\\guidance\\prompt-plan\\promptlib\\')
sys.path.insert(0, '/workspaces/aici/promptlib')


In [2]:
from promptlib import PromptNode
from promptlib import set_model, append, gen, choose, begin, end
from promptlib.models import LLM, TransformersLLM
from promptlib.endpoints import AICI

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
llm = TransformersLLM("gpt2")

pn = PromptNode().set_model(llm)                                          \
    .append("Here is a generated prompt:").gen(max_tokens=10)             \
    .append("Here is another generated prompt. ").gen(max_tokens=10)

aici_steps = pn.build_linear_plan()
print(aici_steps)

{'steps': [{'Model': {'model': 'gpt2'}}, {'Fixed': {'text': 'Here is a generated prompt:'}}, {'Gen': {'max_tokens': 10, 'rx': '.+', 'genargs': {}}}, {'Fixed': {'text': 'Here is another generated prompt. '}}, {'Gen': {'max_tokens': 10, 'rx': '.+', 'genargs': {}}}]}


In [4]:
llm = TransformersLLM("gpt2")

pn0  = PromptNode()

pn = pn0.set_model(llm)                                          \
    .begin(id="block1") \
    .append("Here is a generated prompt. Resp:").gen(max_tokens=10)             \
    .append("Here is another generated prompt. Resp:").gen(max_tokens=10)      \
    .end() \
    .append("Here is a final prompt. Resp:").gen(max_tokens=10)  
    

aici_steps = pn0.build_tree_plan()
print(aici_steps)

{'steps': [{'Model': {'model': 'gpt2'}}, {'block': {'steps': [{'Fixed': {'text': 'Here is a generated prompt. Resp:'}}, {'Gen': {'max_tokens': 10, 'rx': '.+', 'genargs': {}}}, {'Fixed': {'text': 'Here is another generated prompt. Resp:'}}, {'Gen': {'max_tokens': 10, 'rx': '.+', 'genargs': {}}}], 'id': 'block1'}}, {'Fixed': {'text': 'Here is a final prompt. Resp:'}}, {'Gen': {'max_tokens': 10, 'rx': '.+', 'genargs': {}}}]}


In [5]:
llm = TransformersLLM("gpt2")

from promptlib.constraints import DummyCharacterConstraint, OddEvenDigitConstraint

pn0  = PromptNode()

pn = pn0.set_model(llm)                                          \
    .begin(id="block1") \
    .constrain(DummyCharacterConstraint(['h','e','l','o'])) \
    .constrain(OddEvenDigitConstraint()) \
    .append("Here is a generated prompt. Resp:").gen(max_tokens=10)             \
    .append("Here is another generated prompt. Resp:").gen(max_tokens=10)      \
    .end() \
    .append("Here is a final prompt. Resp:").gen(max_tokens=10)  
    

aici_steps = pn0.build_tree_plan()
print(aici_steps)

{'steps': [{'Model': {'model': 'gpt2'}}, {'block': {'steps': [{'constraint': {'type': 'regex', 'rx': '[h|e|l|o]+'}}, {'constraint': {'type': 'regex', 'rx': '^(?:[13579][02468])*[13579]|[02468]([13579]?|(?:[13579][02468])*)$'}}, {'Fixed': {'text': 'Here is a generated prompt. Resp:'}}, {'Gen': {'max_tokens': 10, 'rx': '.+', 'genargs': {}}}, {'Fixed': {'text': 'Here is another generated prompt. Resp:'}}, {'Gen': {'max_tokens': 10, 'rx': '.+', 'genargs': {}}}], 'id': 'block1'}}, {'Fixed': {'text': 'Here is a final prompt. Resp:'}}, {'Gen': {'max_tokens': 10, 'rx': '.+', 'genargs': {}}}]}


In [6]:
ep = AICI(base_url="http://127.0.0.1:8080/v1/", wasm_runner_path="/workspaces/aici/aici_ast_runner/target/opt.wasm")

endpoint = PromptNode().set_endpoint(ep)

pn = endpoint.append("Please answer the following questions: Q: Who is the president of the USA? A:").gen(max_tokens=10)             \
    .append("Q: And who is their vice president? A:").gen(max_tokens=10)

endpoint.run(pn)

upload module... 2713kB -> 2604kB id:09d17084
Please answer the following questions: Q: Who is the president of the USA? A: Joe Biden Q: What is the capital ofQ: And who is their vice president? A: Kamala Harris Q: What is the name of
